<a href="https://colab.research.google.com/github/vitroid/vdwp/blob/main/free_energy_of_cage_occupation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# $f$の導出

(`2017/formulae/enthalpy20151110.tex`より抽出)

まず vdWP 理論の分配関数から。ハイドレート結晶の分配関数は近似的に次のように書ける。

<!-- $$\label{eq:vdWP} -->

$$
\Xi=\exp\left[-\beta A_w^0\right]\prod_k\left[1+\sum_j\exp\left(\beta(\mu_j-f_j^k)\right)\right]^{N_k}\tag{0.1}
$$

ただし、$A_w^0$は空のクラスレートハイドレートを構成する水の自由エネルギー、$k$はケージの種類、$N_k$は単位格子あたりのケージ種 $k$ のケージの個数、$j$はゲスト分子の種類。

(0.1)の対数に$kT$を掛けると Gibbs 自由エネルギーが得られる。

$$
G_w=-kT\ln\Xi=A_w^0-kT\sum_kN_k\ln\left[1+\sum_j\exp\left(\beta(\mu_j-f_j^k)\right)\right]
\tag{0.2}
$$

これを水分子数$N_w$で微分するとクラスレートを構成する水分子の化学ポテンシャルが得られる。

$$
\begin{aligned}
\mu_c&=&\mu_c^0-kT\sum_k\alpha_k\ln\left[1+\sum_j\exp\left(\beta(\mu_j-f_j^k)\right)\right]\\
&=&\mu_c^0+\Delta\mu_c
\end{aligned}
\tag{0.3}
$$

$\alpha_k=N_k/N_w$は単位格子内の水分子の個数に対するケージの個数の比、$\mu_c^0$は空ケージを構成する水分子の化学ポテンシャル、$\Delta\mu_c$はケージにゲストが入ることによる化学ポテンシャル変化分。

$f_j^k$はゲスト種$j$ がケージ種 $k$に含まれる場合のケージ占有自由エネルギーで、以下の細かい項に分解できる。

<!-- $$\label{eq:f} -->

$$
f=f_0+a+c+s\tag{0.4}
$$

$f_0$は積分の中核部分。単原子分子の場合は

$$
f_0=-kT\ln\int\exp\left[-\beta w({\bf r})\right]{\rm d}{\bf r}\tag{0.5}
$$

多原子分子の場合は配向に関する積分が必要になり、

<!-- $$\label{eq:f0mol} -->

$$
f_0=-kT\ln\int_{\bf \Omega}\int_{\bf r}\exp\left[-\beta w({\bf r},{\bf \Omega})\right]{\rm d}{\bf r}{\rm d}{\bf \Omega}\tag{0.6}
$$

(さらに$\Omega$を具体的に分解する)

$a$は質量項。

$$
a=-\frac{3kT\ln(m/2\pi\beta\hbar^2)}{2}\tag{0.7}
$$

$c$は慣性モーメント項。(直線分子と剛体分子のみ)。 直線分子の場合

$$
c=-kT\ln(I/2\pi\beta\hbar^2)\tag{0.8}
$$

剛体分子の場合

$$
c=-\frac{kT}{2}\sum_{j=1}^3\ln(I_j/2\pi\beta\hbar^2)\tag{0.9}
$$

$s$は分子の対称性からの寄与。

$$
s=kT\ln\sigma\tag{0.10}
$$

$\sigma$は分子の回転対称性からくる項。

$\mu$は分解状態でのゲスト分子の化学ポテンシャルであり、液体か気体か水溶液かで計算の方法が異なる。


## 2. 計算の高速化

この一連の計算のなかで、もっとも計算コストが高いのは、(0.6)の分配関数に現れる積分である。これに関しては、`cage_integral/README.md`を参照。


In [2]:
import vdwp.chempot as chempot
from cage_integral import histo2f, histo, molecule

# 分子の形と相互作用の情報を読み込む
with open("cage_integral/DEFR", encoding="utf-8") as file:
    moldict = molecule.loadInfo(file)

# ゲスト分子のラベルを入力する
label = input("Guest label (e.g. LJME____)")
guest = moldict[label]

temperature = 273.15

for cage in 12, 14, 16:

    # ヒストグラムファイルを読み込む
    histofile = f"cage_integral/{label}.{cage}hedra.histo"
    histogram = histo.loadAHisto(open(histofile))

    # ヒストグラムが存在する場合
    if histogram != None:
        # ケージ占有自由エネルギーを計算する
        print(
            cage,
            # histo2f.fvalue(histogram, temperature),
            histo2f.fvalue(histogram, temperature)
            + chempot.molecular_chemical_potential_corrections(
                temperature, guest.mass, guest.symm, guest.moi
            ),
        )

12 -25.72669854539032
14 -28.98053517251452
16 -29.553031016967765
